In [117]:
import pandas as pd
import getpass
import os
username = getpass.getuser()
Year = 2024
Month = 3

# Define the file path
file_path = f'C:/Users/{username}/Box/REAL/IAR/Working Folder/MSA/table_5_sales.csv'

# Read the CSV file into a DataFrame
salesdata = pd.read_csv(file_path)

# Define the file path
file_path = f'C:/Users/{username}/Box/REAL/IAR/Working Folder/MSA/table_4_price.csv'

# Read the CSV file into a DataFrame
pricedata = pd.read_csv(file_path)

# Define the file path
file_path = rf'C:/Users/{username}/Box/REAL/IAR/Working Folder/MSA/working files/MSA price strati/PriceStrataTable.csv'
stratadata = pd.read_csv(file_path)

# Define the file path
file_path = rf'C:/Users/{username}/Box/REAL/IAR/Working Folder/MSA/working files/MSA quarter supply/SupplyTable.csv'
supplydata = pd.read_csv(file_path)


In [42]:
import pandas as pd
import getpass
username = getpass.getuser()

import re

def get_previous_quarter(quarter, year):
    """
    Calculate the previous quarter and year.

    :param quarter: The current quarter (e.g., 'Q1', 'Q2', 'Q3', 'Q4')
    :param year: The current year (e.g., 2023)
    :return: A tuple containing the previous quarter and year
    """
    # Extract the quarter number
    quarter_number = int(re.search(r'\d+', quarter).group())
    previous_year = year

    # Determine the previous quarter
    if quarter_number == 1:
        previous_quarter = 'Q4'
        previous_year -= 1  # Decrement the year if the current quarter is Q1
    else:
        previous_quarter = f'Q{quarter_number - 1}'

    return previous_quarter, previous_year

def count_unique_positive_values(data, area_column, value_column):
    """
    Counts the number of unique values in the specified 'area_column' where the values in 'value_column' are positive.
    This function does not modify the input DataFrame.

    :param data: DataFrame containing the data
    :param area_column: The name of the column to count unique values from
    :param value_column: The name of the column to check for positive values
    :return: The count of unique values in 'area_column' with positive values in 'value_column'
    """
    # Create a copy of the relevant columns to avoid modifying the original DataFrame
    temp_data = data[[area_column, value_column]].copy()

    # Convert the value column to numeric, ignoring non-numeric data
    temp_data[value_column] = pd.to_numeric(temp_data[value_column].str.replace('%', ''), errors='coerce')

    # Count the unique values
    return temp_data[temp_data[value_column] > 0][area_column].nunique()


def get_cell_value_sales(data, area, forecast1, column_name):
    """
    Returns the value from the specified column where 'forecast1' and 'area' match the given criteria.

    :param data: DataFrame containing the data
    :param area: The area to match
    :param forecast1: The forecast1 value to match
    :param column_name: The column from which to extract the value
    :return: The extracted value or None if no match is found
    """
    filtered_data = data[(data['forecast1'] == forecast1) & (data['area'] == area)]
    if not filtered_data.empty:
        return filtered_data[column_name].iloc[0]
    else:
        return None

def get_cell_value_price(data, area, summary0, column_name):
    """
    Returns the value from the specified column where 'forecast1' and 'area' match the given criteria.

    :param data: DataFrame containing the data
    :param area: The area to match
    :param forecast1: The forecast1 value to match
    :param column_name: The column from which to extract the value
    :return: The extracted value or None if no match is found
    """
    filtered_data = data[(data['summary0'] == summary0) & (data['area'] == area)]
    if not filtered_data.empty:
        return filtered_data[column_name].iloc[0]
    else:
        return None

def interpret_cell_value(cell_value):
    """
    Returns 'increased' or 'decreased' based on the sign of the cell value.

    :param cell_value: The cell value as a string, potentially including a percentage sign.
    :return: 'increased' if the value is positive, 'decreased' if the value is negative, or None if the value is not a valid number.
    """
    try:
        # Remove percentage sign and convert to float
        numeric_value = float(cell_value.replace('%', ''))
        return 'increased' if numeric_value > 0 else 'decreased'
    except ValueError:
        # If the conversion fails, return None
        return None


In [43]:
def get_sales_values(data, area, forecast1, columns):
    """
    Returns the values from specified columns where 'forecast1' and 'area' match the given criteria.

    :param data: DataFrame containing the data
    :param area: The area to match
    :param forecast1: The forecast1 value to match
    :param columns: List of columns from which to extract the values
    :return: A dictionary with column names as keys and extracted values as values
    """
    results = {}
    for column in columns:
        if column in data.columns:
            filtered_data = data[(data['forecast1'] == forecast1) & (data['area'] == area)]
            if not filtered_data.empty and column in filtered_data.columns:
                results[column] = filtered_data[column].iloc[0]
            else:
                results[column] = "No matching data found."
        else:
            results[column] = f"Column '{column}' not found in the data."
    return results

def get_annually1_value(data, area, forecast1, column_name):
    """
    Returns the value from the specified column where 'forecast1' and 'area' match the given criteria.

    :param data: DataFrame containing the data
    :param area: The area to match
    :param forecast1: The forecast1 value to match
    :param column_name: The column from which to extract the value
    :return: The extracted value or a message if no match is found
    """
    if column_name in data.columns:
        filtered_data = data[(data['forecast1'] == forecast1) & (data['area'] == area)]
        if not filtered_data.empty:
            return filtered_data[column_name].iloc[0]
        else:
            return "No matching data found."
    else:
        return f"Column '{column_name}' not found in the data."




Overview

In [44]:
def create_sales_paragraph(data, area, current_year, current_quarter):
    """
    Creates a paragraph with updated sales information based on the provided data.

    :param data: DataFrame containing the sales data
    :param area: The area (e.g., 'Illinois') to focus the analysis on
    :param year: The year of interest
    :param current_quarter: The current quarter (e.g., 'Q3')
    :param next_quarter: The next quarter (e.g., 'Q4')
    :return: A paragraph with updated sales information
    """

    prev_quarter, prev_year = get_previous_quarter(current_quarter, current_year)
    # Get the number of MSAs with positive sales growth in the current and next quarters
    positive_growth_current = count_unique_positive_values(data, 'area', 'salesa0')
    positive_growth_next = count_unique_positive_values(data, 'area', 'salesa10')

    # Create the paragraph
    paragraph = (
        f"In the {prev_quarter} of {prev_year}, overall sales in {area} decreased compared to last year. "
        f"Positive growth was recorded in {positive_growth_current} of the MSAs. "
        f"For the {current_quarter} of {current_year}, overall sales in {area} are forecast to decrease. "
        f"{positive_growth_next} out of ten MSAs are forecast to experience an increase in their sales."
    )

    return paragraph

def create_price_paragraph(data, area, current_quarter, next_quarter, year):
    """
    Creates a paragraph with updated price information based on the provided data.

    :param data: DataFrame containing the price data
    :param area: The area (e.g., 'Illinois') to focus the analysis on
    :param current_quarter: The current quarter (e.g., 'Third Quarter')
    :param next_quarter: The next quarter (e.g., 'Fourth Quarter')
    :param year: The year of interest
    :return: A paragraph with updated price information
    """
    # Get the number of MSAs with increasing median prices in the current and next quarters
    increasing_prices_current = count_unique_positive_values(data, 'area', 'annually0')-1
    increasing_prices_next = count_unique_positive_values(data, 'area', 'annually1')-1

    # Create the paragraph
    paragraph = (
        f"Increasing median prices for {area} are found in {increasing_prices_current} of the 10 MSAs in the {current_quarter}. "
        f"For the {next_quarter} of {year}, {increasing_prices_next} MSAs are projected to experience a positive growth in median price."
    )

    return paragraph

In [45]:
get_previous_quarter('Q2',2024)

('Q1', 2024)

In [55]:
OverViewSales = create_sales_paragraph(salesdata, 'Illinois', 2024, 'Q2')
OverViewPrice = create_price_paragraph(pricedata, 'Illinois', 'First Quarter', 'Second Quarter', 2024)
Draft = (OverViewSales,OverViewPrice)

In [57]:
os.chdir(rf'C:\\Users\\Keitaro Ninomiya\\Box\\IAR_nino\\jc{Year}{Month}\\Draft\\QuarterlyReport')
textfile = open("Overview.txt", "w")
for element in Draft:
    textfile.write(element + "\n")
textfile.close()

Housing Market Condition

In [58]:
# Redefining the function using the correct price data and column
def create_sales_paragraph_with_positive_change(sales_data, area, year, current_quarter, sales_column):
    """
    Creates a paragraph with sales information and details on areas with positive changes in price.

    :param sales_data: DataFrame containing the sales data
    :param price_data: DataFrame containing the price data
    :param area: The area (e.g., 'Illinois') for the analysis
    :param year: The year of interest
    :param current_quarter: The current quarter (e.g., 'Third Quarter')
    :param sales_column: The sales column name to check for positive values
    :param price_column: The price column name to identify positive changes in price
    :return: A paragraph with sales information and positive change details
    """
    # Count of MSAs with annual increase in sales
    count_positive_sales = count_unique_positive_values(sales_data, 'area', sales_column)

    # Overall sales change for Illinois
    overall_sales_change = get_cell_value_sales(sales_data, area, '3M', sales_column)

    # MSAs with positive changes in price
    positive_changes = positive_price_area_sequence_full_names(sales_data, '3M', sales_column)

    # Creating the paragraph
    paragraph = (
        f"In the {current_quarter} of {year}, {area} experienced a decrease in sales, "
        f"and {count_positive_sales} MSAs also experienced an annual increase in sales with varying degrees. "
        f"Overall sales for {area} decreased by {overall_sales_change} compared to a year ago. "
        f"The MSAs with positive changes in price were: {positive_changes}."
    )

    return paragraph



In [59]:
def positive_area_sequence(data, summary_criteria, positive_value_column):
    """
    Generates a sequence of area names with positive values in a specified column, 
    formatted as 'Area (Value)', for rows matching the summary criteria.

    :param data: DataFrame containing the data
    :param summary_criteria: The value to filter the 'summary0' column
    :param positive_value_column: The column to check for positive values
    :return: A string containing the formatted sequence of area names with positive values
    """
    # Filter data based on summary criteria
    filtered_data = data[data['summary0'] == summary_criteria]

    # Find areas with positive values in the specified column
    positive_areas = filtered_data[filtered_data[positive_value_column].str.replace('%', '').astype(float) > 0]

    # Generate the sequence of texts
    positive_areas_list = [f"{row['area']} ({row[positive_value_column]})" for _, row in positive_areas.iterrows()]
    return ', '.join(positive_areas_list)



In [60]:
# Redefining the function using the correct price data and column
def create_price_paragraph_with_positive_change(pricedata, area, year, current_quarter, price_column):
    """
    Creates a paragraph with sales information and details on areas with positive changes in price.

    :param sales_data: DataFrame containing the sales data
    :param price_data: DataFrame containing the price data
    :param area: The area (e.g., 'Illinois') for the analysis
    :param year: The year of interest
    :param current_quarter: The current quarter (e.g., 'Third Quarter')
    :param sales_column: The sales column name to check for positive values
    :param price_column: The price column name to identify positive changes in price
    :return: A paragraph with sales information and positive change details
    """
    # Count of MSAs with annual increase in sales
    count_positive_price = count_unique_positive_values(pricedata, 'area', price_column)

    # Overall sales change for Illinois
    overall_price_change = get_cell_value_price(pricedata, area, '3M', price_column)

    # MSAs with positive changes in price
    positive_changes = positive_price_area_sequence_full_names(pricedata, '3M', price_column)

    # Creating the paragraph
    paragraph = (
        f"In the {current_quarter} of {year}, {area} experienced a decrease in sales, "
        f"and {count_positive_price} MSAs also experienced an annual increase in sales with varying degrees. "
        f"Overall prices for {area} decreased by {overall_price_change} compared to a year ago. "
        f"The MSAs with positive changes in price were: {positive_changes}."
    )

    return paragraph


In [61]:
# Mapping of abbreviations to full county names
area_mapping = {
    'IL': 'Illinois',
    'BL': 'Bloomington',
    'CU': 'Champaign-Urbana',
    'CH': 'Chicago',
    'DRM': 'Davenport-Moline-Rock Island',
    'DE': 'Decatur',
    'KK': 'Kankakee',
    'ME': 'Metro-East',
    'PP': 'Peoria-Pekin',
    'RO': 'Rockford',
    'SP': 'Springfield'
}

def positive_price_area_sequence_full_names(data, summary_criteria, positive_value_column):
    """
    Generates a sequence of area full names with positive values in a specified column (e.g., 'annually0'), 
    formatted as 'Area (Value)', for rows matching the summary criteria.

    :param data: DataFrame containing the price data
    :param summary_criteria: The value to filter the 'summary0' column
    :param positive_value_column: The column to check for positive values in price data
    :return: A string containing the formatted sequence of area full names with positive values
    """
    # Filter data based on summary criteria
    filtered_data = data[data['summary0'] == summary_criteria]

    # Find areas with positive values in the specified column
    positive_areas = filtered_data[filtered_data[positive_value_column].str.replace('%', '').astype(float) > 0]

    # Generate the sequence of texts with full area names
    positive_areas_list = [
        f"{area_mapping.get(row['area'], row['area'])} ({row[positive_value_column]})"
        for _, row in positive_areas.iterrows()
    ]
    return ', '.join(positive_areas_list)


In [31]:
import pandas as pd

def generate_supply_paragraph(data, area, msa_crosswalk):
    """
    Generates a paragraph summarizing the supply changes in housing across MSAs and the area,
    focusing on the 'Overall' value in the 'rangestr' column.

    :param data: DataFrame containing the supply data
    :param area: The area (e.g., 'Illinois') for the analysis
    :param msa_crosswalk: Dictionary mapping MSA IDs to MSA names
    :return: A paragraph summarizing the supply changes
    """
    # Get overall data for the specified area
    overall_area_data = data[(data['MSA'] == area) & (data['rangestr'] == "Overall")]

    # Calculate overall supply change for the area
    if not overall_area_data.empty:
        overall_supply_thisq = round(overall_area_data['thisq'].iloc[0], 1)
        overall_supply_lastq = round(overall_area_data['lastq'].iloc[0], 1)
        overall_supply_change = f"The overall quarter’s supply for {area} is {overall_supply_thisq}, decreasing from {overall_supply_lastq} of a year ago."
    else:
        overall_supply_change = f"No overall supply data available for {area}."

    # Filter for MSA data and consider only 'Overall' in rangestr
    msa_data = data[data['rangestr'] == "Overall"]
    msa_data['MSA'] = msa_data['MSA'].apply(lambda x: msa_crosswalk.get(int(x), x) if x.isdigit() else x)

    # Identify positive, negative, and no changes in supply for each MSA
    positive_changes, negative_changes, no_changes = [], [], []
    for _, row in msa_data.iterrows():
        if row['MSA'] == area:  # Skip the overall area row
            continue
        thisq_rounded = round(row['thisq'], 1)
        lastq_rounded = round(row['lastq'], 1)
        msa_name = str(row['MSA'])
        if thisq_rounded > lastq_rounded:
            positive_changes.append(f"{msa_name} (to {thisq_rounded} from {lastq_rounded})")
        elif thisq_rounded < lastq_rounded:
            negative_changes.append(f"{msa_name} (to {thisq_rounded} from {lastq_rounded})")
        else:
            no_changes.append(msa_name)

    # Constructing the paragraph with conditional sentences
    parts = [overall_supply_change]
    if positive_changes:
        parts.append(f"One MSA experienced positive changes: {', '.join(positive_changes)}.")
    if negative_changes:
        parts.append(f"{len(negative_changes)} MSAs experienced negative changes: {', '.join(negative_changes)}.")
    if no_changes:
        parts.append(f"{len(no_changes)} MSAs experienced no changes: {', '.join(no_changes)}.")

    paragraph = ' '.join(parts)
    return paragraph

# MSA Crosswalk
msa_crosswalk = {
    1: "Bloomington-Normal",
    2: "Champaign-Urbana",
    3: "Chicago PMSA",
    4: "Davenport-Rock Island-Moline",
    5: "Decatur",
    6: "Kankakee",
    7: "Metro-East",
    8: "Peoria-Pekin",
    9: "Rockford",
    10: "Springfield"
}

C:\Temp\ipykernel_22124\1223333068.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msa_data['MSA'] = msa_data['MSA'].apply(lambda x: msa_crosswalk.get(int(x), x) if x.isdigit() else x)


'The overall quarter’s supply for Illinois is 0.5, decreasing from 0.7 of a year ago. 9 MSAs experienced negative changes: Bloomington-Normal (to 0.1 from 0.2), Springfield (to 0.3 from 0.4), Champaign-Urbana (to 0.3 from 0.6), Chicago PMSA (to 0.4 from 0.6), Davenport-Rock Island-Moline (to 0.4 from 0.5), Decatur (to 0.3 from 0.5), Kankakee (to 0.6 from 0.7), Peoria-Pekin (to 0.4 from 0.5), Rockford (to 0.3 from 0.5). 1 MSAs experienced no changes: Metro-East.'

In [69]:
# Generating the paragraph
HousingMarketSales = create_sales_paragraph_with_positive_change(salesdata, 'Illinois', 2024, 'First Quarter', 'salesa0')
HousingMarketPrices = create_price_paragraph_with_positive_change(pricedata, 'Illinois', 2024, 'First Quarter', 'annually0')
HousingMarketSupply = generate_supply_paragraph(supplydata, 'Illinois', msa_crosswalk)

Draft = (HousingMarketSales,HousingMarketPrices,HousingMarketSupply)

os.chdir(rf'C:\\Users\\Keitaro Ninomiya\\Box\\IAR_nino\\jc{Year}{Month}\\Draft\\QuarterlyReport')
textfile = open("HousingMarketConditions.txt", "w")
for element in Draft:
    textfile.write(element + "\n")
textfile.close()

C:\Temp\ipykernel_22124\1223333068.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msa_data['MSA'] = msa_data['MSA'].apply(lambda x: msa_crosswalk.get(int(x), x) if x.isdigit() else x)


Housing Market Forecasts

In [32]:
def positive_sales_areas(data, summary_criteria, value_column, area_mapping):
    """
    Generates a sequence of area full names with positive values in a specified column, 
    formatted as 'Area', for rows matching the summary criteria.

    :param data: DataFrame containing the sales data
    :param summary_criteria: The value to filter the 'summary0' column
    :param value_column: The column to check for positive values
    :param area_mapping: Dictionary mapping abbreviations to full area names
    :return: A string containing the formatted sequence of area full names with positive values
    """
    # Filter data based on summary criteria
    filtered_data = data[data['summary0'] == summary_criteria]

    # Find areas with positive values in the specified column
    positive_areas = filtered_data[filtered_data[value_column].str.replace('%', '').astype(float) > 0]

    # Generate the sequence of texts with full area names
    positive_areas_list = [
        f"{area_mapping.get(row['area'], row['area'])}"
        for _, row in positive_areas.iterrows()
    ]
    return ', '.join(positive_areas_list)

# Generating the sequence with full area names for areas with positive values in salesa0
positive_sales_areas_sequence = positive_sales_areas(salesdata, '3M', 'salesa0', area_mapping)
positive_sales_areas_sequence

''

In [74]:
def create_forecast_paragraph(sales_data, area, year, quarter, area_mapping):
    """
    Creates a paragraph with sales forecast information.

    :param sales_data: DataFrame containing the sales data
    :param area: The area (e.g., 'Illinois') for the analysis
    :param year: The year of interest
    :param quarter: The quarter for the forecast (e.g., 'Fourth Quarter')
    :param area_mapping: Dictionary mapping abbreviations to full area names
    :return: A paragraph with sales forecast information
    """
    # Overall sales forecast change for the area
    sales_forecast_change = get_sales_values(sales_data, area, '3M', ['salesa10', 'salesa11'])

    # Count of MSAs with forecasted positive sales
    count_positive_forecast = count_unique_positive_values(sales_data, 'area', 'salesa0')

    # MSAs with forecasted positive sales
    positive_forecast_areas = positive_sales_areas(salesdata, '3M', 'salesa0', area_mapping)

    # Creating the paragraph
    paragraph = (
        f"Table 5 provides the sales forecast for the {quarter} of {year}. "
        f"The overall sales in {area} are forecasted to decrease by {sales_forecast_change['salesa10']} to {sales_forecast_change['salesa11']} compared to a year ago. "
        f"{count_positive_forecast} MSAs are forecasted to experience a positive increase in sales: {positive_forecast_areas}."
    )

    return paragraph

In [73]:
# Defining the positive_price_areas function
def positive_price_areas_with_percentage(data, summary_criteria, value_column, area_mapping):
    """
    Generates a sequence of area full names with positive values in a specified column, 
    formatted as 'Area (Value)', for rows matching the summary criteria.

    :param data: DataFrame containing the price data
    :param summary_criteria: The value to filter the 'summary0' column
    :param value_column: The column to check for positive values
    :param area_mapping: Dictionary mapping abbreviations to full area names
    :return: A string containing the formatted sequence of area full names with positive values and percentages
    """
    # Filter data based on summary criteria
    filtered_data = data[data['summary0'] == summary_criteria]

    # Find areas with positive values in the specified column
    positive_areas = filtered_data[filtered_data[value_column].str.replace('%', '').astype(float) > 0]

    # Generate the sequence of texts with full area names and percentages
    positive_areas_list = [
        f"{area_mapping.get(row['area'], row['area'])} ({row[value_column]})"
        for _, row in positive_areas.iterrows()
    ]
    return ', '.join(positive_areas_list)


# Defining a similar function for price data
def create_price_forecast_paragraph(price_data, area, year, quarter, area_mapping):
    """
    Creates a paragraph with price forecast information.

    :param price_data: DataFrame containing the price data
    :param area: The area (e.g., 'Illinois') for the analysis
    :param year: The year of interest
    :param quarter: The quarter for the forecast (e.g., 'Fourth Quarter')
    :param area_mapping: Dictionary mapping abbreviations to full area names
    :return: A paragraph with price forecast information
    """
    # Positive price areas
    positive_price_areas_list = positive_price_areas_with_percentage(price_data, '3M', 'annually0', area_mapping)

    # Creating the paragraph
    paragraph = (
        f"Table 4 provides the price forecast for the {quarter} of {year}. "
        f"The MSAs expected to experience a positive growth in price are: {positive_price_areas_list}."
    )

    return paragraph



'Table 4 provides the price forecast for the Fourth Quarter of 2023. The MSAs expected to experience a positive growth in price are: Illinois (7.6%), Bloomington (10.2%), Champaign-Urbana (14.3%), Chicago (6.8%), Davenport-Moline-Rock Island (6.7%), Decatur (7.4%), Kankakee (9.3%), Metro-East (5.9%), Peoria-Pekin (11.1%), Rockford (9.4%), Springfield (3.0%).'

In [76]:
ForecastSales = create_forecast_paragraph(salesdata, 'IL', 2023, 'Fourth Quarter', area_mapping)
ForecastPrice = create_price_forecast_paragraph(pricedata, 'IL', 2023, 'Fourth Quarter', area_mapping)
Draft = (ForecastPrice,ForecastSales)

os.chdir(rf'C:\\Users\\Keitaro Ninomiya\\Box\\IAR_nino\\jc{Year}{Month}\\Draft\\QuarterlyReport')
textfile = open("HousingMarketForecast.txt", "w")
for element in Draft:
    textfile.write(element + "\n")
textfile.close()

MSAs

In [118]:
def generate_sentences(pricedata, salesdata,area, current_quarter, current_year):
    """
    Generate sentences based on the given data, area, quarter, and year.

    :param pricedata: DataFrame containing price data
    :param salesdata: DataFrame containing sales data
    :param area: The area for which to generate sentences
    :param current_quarter: The current quarter
    :param current_year: The current year
    :return: A tuple of generated sentences
    """
    # Calculate the previous quarter and year
    prev_quarter, prev_year = get_previous_quarter(current_quarter, current_year)

    # Modify the forecast1 values to match the expected format in the data
    forecast1_current = f"{current_quarter} {current_year}"
    forecast1_previous = f"{prev_quarter} {prev_year}"

    # Generating the sentences
    sentence1 = f"Median price in {forecast1_current} is forecasted to increase by an annual rate of {get_annually1_value(pricedata, area, '3M Average', 'annually1')}."
    sales_values = get_sales_values(salesdata, area, '3M', ['salesa10', 'salesa11'])
    sentence2 = f"The sales forecast indicates a negative trend with annual rates of change between {sales_values['salesa10']} to {sales_values['salesa11']} in {forecast1_current}."
    sentence3 = f"Median prices in {forecast1_previous} increased by {get_cell_value_price(pricedata, area, '3M', 'annually0')} from a year ago."
    sentence4 = f"Sales volume in {forecast1_previous} was {get_cell_value_sales(salesdata, area, '3M', 'salesa0')} lower than a year ago."

    return sentence1, sentence2, sentence3, sentence4

def generate_market_share_change_paragraph(data, msa_id, msa_crosswalk, current_quarter, current_year):
    """
    Generates a sentence about the market share change for the price range with the largest difference
    in a specific MSA. Handles MSA IDs as either strings or integers.

    :param data: DataFrame containing the price range data
    :param msa_id: ID of the MSA of interest (can be string or integer)
    :param msa_crosswalk: Dictionary mapping MSA IDs to MSA names
    :return: A sentence about the market share change
    """
    # Convert MSA ID to string if it's an integer
    msa_id_str = str(msa_id)

    # Filter data for the specified MSA
    msa_data = data[data['MSA'].astype(str) == msa_id_str]

    # If no data found for the MSA, return a message
    if msa_data.empty:
        return f"No data available for MSA with ID {msa_id}."

    # Calculate the difference between 'thisq' and 'lasty'
    msa_data['difference'] = msa_data['thisq'] - msa_data['lasty']

    # Find the row with the largest difference
    largest_change_row = data.iloc[msa_data['difference'].abs().idxmax()]

    # Generate the sentence
    price_range = largest_change_row['rangestr']
    thisq_value = round(largest_change_row['thisq'], 1)
    lasty_value = round(largest_change_row['lasty'], 1)
    msa_name = msa_crosswalk.get(msa_id, f"MSA {msa_id}")
    sentence = f"In {current_quarter} {current_year}, {msa_name} market shares of homes priced at {price_range} experienced the largest change, increasing to {thisq_value}% from {lasty_value}% a year ago."

    return sentence

def generate_overall_supply_sentence_for_msa(data, msa_id, msa_crosswalk, current_quarter, current_year):
    """
    Generates a sentence about the overall quarter’s supply for a specific MSA, 
    comparing 'thisq' and 'lasty' columns.

    :param data: DataFrame containing the supply data
    :param msa_id: ID of the MSA of interest
    :param msa_crosswalk: Dictionary mapping MSA IDs to MSA names
    :return: A sentence about the overall supply change for the specified MSA
    """
    # Convert MSA ID to string for comparison
    msa_id_str = str(msa_id)

    # Filter data for the specified MSA and 'Overall' in rangestr
    msa_data = data[(data['MSA'].astype(str) == msa_id_str) & (data['rangestr'] == "Overall")]

    # Ensure there's data for the specified MSA
    if msa_data.empty:
        return f"No overall supply data available for MSA with ID {msa_id}."

    # Extract current and last year's supply values
    thisq_value = round(msa_data['thisq'].iloc[0], 1)
    lasty_value = round(msa_data['lasty'].iloc[0], 1)
    msa_name = msa_crosswalk.get(msa_id, f"MSA {msa_id}")

    # Constructing the sentence
    sentence = f"By {current_quarter} {current_year}, the overall quarter’s supply for {msa_name} is {thisq_value} quarters, decreasing from {lasty_value} a year ago."

    return sentence

def find_key_by_value(my_dict, search_value):
    for key, value in my_dict.items():
        if value == search_value:
            return key
    return None  # If no key found for the given value

In [91]:
area_mapping = {
    'IL': 'Illinois',
    'BL': 'Bloomington',
    'CU': 'Champaign-Urbana',
    'CH': 'Chicago',
    'DRM': 'Davenport-Moline-Rock Island',
    'DE': 'Decatur',
    'KK': 'Kankakee',
    'ME': 'Metro-East',
    'PP': 'Peoria-Pekin',
    'RO': 'Rockford',
    'SP': 'Springfield'
}

# MSA Crosswalk
msa_crosswalk = {
    0: "Illinois",
    1: "Bloomington-Normal",
    2: "Champaign-Urbana",
    3: "Chicago PMSA",
    4: "Davenport-Rock Island-Moline",
    5: "Decatur",
    6: "Kankakee",
    7: "Metro-East",
    8: "Peoria-Pekin",
    9: "Rockford",
    10: "Springfield"
}


In [122]:
current_quarter = 'Q2'
current_year = 2024
previous_quarter = get_previous_quarter('Q2',2024)[0]
previous_year = get_previous_quarter('Q2',2024)[1]

os.chdir(rf'C:\\Users\\Keitaro Ninomiya\\Box\\IAR_nino\\jc{Year}{Month}\\Draft\\QuarterlyReport')

for area in area_mapping.keys():
    generated_sentences = generate_sentences(pricedata, salesdata, area, current_quarter, current_year)
    
    MSAname = area_mapping[area]

    MSAID = find_key_by_value(msa_crosswalk, MSAname)
    Strata = generate_market_share_change_paragraph(stratadata, MSAID, msa_crosswalk, previous_quarter, previous_year)
    Supply = generate_overall_supply_sentence_for_msa(supplydata, MSAID, msa_crosswalk, previous_quarter, previous_year)

    Draft = generated_sentences + (Strata,Supply,)

    textfile = open(rf"HousingMarketMSA_{area}.txt", "w")
    for element in Draft:
        textfile.write(element + "\n")
    textfile.close()

    


C:\Temp\ipykernel_22124\2498852554.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msa_data['difference'] = msa_data['thisq'] - msa_data['lasty']
C:\Temp\ipykernel_22124\2498852554.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msa_data['difference'] = msa_data['thisq'] - msa_data['lasty']
C:\Temp\ipykernel_22124\2498852554.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 